<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Run-Simulations-(non-OOP)" data-toc-modified-id="Run-Simulations-(non-OOP)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Run Simulations (non-OOP)</a></span><ul class="toc-item"><li><span><a href="#Creating-the-game-mechanics" data-toc-modified-id="Creating-the-game-mechanics-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Creating the game mechanics</a></span></li><li><span><a href="#Simulating-the-game" data-toc-modified-id="Simulating-the-game-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Simulating the game</a></span></li><li><span><a href="#Run-many-games" data-toc-modified-id="Run-many-games-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Run many games</a></span></li><li><span><a href="#Run-many-kinds-of-games" data-toc-modified-id="Run-many-kinds-of-games-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Run many kinds of games</a></span></li><li><span><a href="#Optional:-Parallel-Processing" data-toc-modified-id="Optional:-Parallel-Processing-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Optional: Parallel Processing</a></span></li></ul></li></ul></div>

Checkout the game on this video: https://www.youtube.com/watch?v=EuwNgNiC4BY

# Run Simulations (non-OOP)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(27)

## Creating the game mechanics

In [ ]:
def winner(player1_choice, player2_choice):
    '''
    Uses rock-paper-scissors logic & returns winner ('player1'/'player2') or 
    'draw'.
    '''
    if player1_choice == player2_choice:
        return 'draw'
    elif player1_choice == 0:
        if player2_choice == 1:
            return 'player2'
        else:
            return 'player1'
    elif player1_choice == 1:
        if player2_choice == 2:
            return 'player2'
        else:
            return 'player1'
    else: #player1_choice == 2:
        if player2_choice == 0:
            return 'player2'
        else:
            return 'player1'        

def battle():
    '''
    Returns 0 or 1 for winner (player 1 or 2). Will run battle until a winner.
    '''
    
    # Play game
    player1 = np.random.randint(0,3)
    player2 = np.random.randint(0,3)
    battle_winner = winner(player1, player2)
    
    # Play until
    while(battle_winner == 'draw'):
        player1 = np.random.randint(0,3)
        player2 = np.random.randint(0,3)
        battle_winner = winner(player1, player2)
        
    #
    return 0 if battle_winner == 'player1' else 1

    

## Simulating the game

In [ ]:
def simulate_game(n_steps=9, printout=False):
    # Start each team's player on opposite sides of the chain
    team1_pos = 0
    team2_pos = n_steps

    # Keep track of how many rounds (time-steps)
    rounds = 0

    # Iterate while neither player is at their respective end/finish
    while(team1_pos < n_steps and team2_pos > 0):
        # For debugging
        if printout:
            print(f'{rounds}: {team1_pos},{team2_pos}')

            
        # TODO: Note with even n_steps, players can "skip" over another
        # Need to consider if other player is in way before moving
        
        # Advance players 
        rounds += 1
        team1_pos += 1
        team2_pos -= 1

        # Check if there's about to be a battle!!
        if team1_pos >= team2_pos:
            if printout:
                print('battle')
            if battle() == 0: # Team2 resets
                team2_pos = n_steps
            else: # Team1 resets
                team1_pos = 0

    # For debugging
    if printout:
        print()
        print(f'Winner: {"Team 2" if team1_pos==0 else "Team 1"}')
              
    return rounds

In [ ]:
y = [simulate_game(printout=False) for _ in range(500)]

In [ ]:
# Simulate game many times and observe the distribution
y = [simulate_game(19,printout=False) for _ in range(100)]
plt.hist(y)

## Run many games

In [ ]:
def run_n_simulations(n_steps, n_sims=150, printout=False):
    # Simulate many times
    data=[]
    for _ in range(n_sims):
        data.append(simulate_game(n_steps))
    return n_steps, np.mean(data)

In [ ]:
# Example
run_n_simulations(9)

## Run many kinds of games

In [ ]:
# Record the mean of each game variation
mean_data = []

start = 9     # how big the game's chain is
n_types = 15  # how many different types of games (chain-length)
n_skips = 10  # start + n_skips ⇒ gives new game type (chain-lenght)

for i in range(n_types):
    # Print out each game type
    print(f'#{i}: Game Size:{i*n_skips+start}'')
          
    data = []
    # Simulate many times for this game type
    for _ in range(250):
        data.append(simulate_game(start+i*n_skips))
    mean_data.append(np.mean(data))
          
    # Print out average game length
    print('\t Mean Game Length (rounds): {mean_data[i]}')
          
    
plt.scatter(range(start,start+n_types*n_skips,n_skips),mean_data);

In [ ]:
# Simplified version (no printouts)
start = 9
n_types = 20
n_skips = 20
game_sizes=[]
avg_n_rounds=[]


for steps in range(start, start+n_types*n_skips, n_skips):
    x,y = run_n_simulations(steps)
    game_sizes.append(x)
    avg_n_rounds.append(y)


plt.scatter(game_sizes, avg_n_rounds);

## Optional: Parallel Processing

In [ ]:
import multiprocessing as mp
num_workers = mp.cpu_count()

start = 9
n_types = 20
n_skips = 20
out1=[]
out2=[]

pool = mp.Pool(num_workers)
for task in range(start, start+n_types*n_skips, n_skips):
    res = pool.apply_async(run_n_simulations, args=(task,))
    x,y = res.get(timeout=10) # time out fail-safe for task
    out1.append(x)
    out2.append(y)

pool.close()
pool.join()

plt.scatter(out1, out2);